In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')


********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [5]:
features_list =['Num_Enq_non_BANK_last_12M',
'Num_Enq_all_loans_last_9M_3M',
'No of enq in past 90days',
'num_PERSONAL LOAN_enq_last_36M',
                
'Num_of_PERSONAL LOAN_opened_last9M',
'Num_of_PERSONAL LOAN_opened_last12M',
'highest_sanction_amount_credit_cards',            
'num_CONSUMER LOAN_enq_last_36M',


'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_outstanding_Balance_PERSONAL LOAN_opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_12M',


'Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M',
'Utilization_all',
'Age_of_oldest_open_account',
'total_emi',
'percentage_ever_delinquent',
                
'num_CONSUMER LOAN_enq_last_6M_36M',


    
#ID Features
'bankcard_vel_2',
'g406s',
'month_year',
'90_in_24mob']
              

In [6]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) 
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3021811/1249756386.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3021811/1249756386.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 19), (15604, 19), (23406,), (15604,))

In [9]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 19) (4682, 19) (15604, 19)


In [10]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [11]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7.5,
    'alpha':5.5,
    'gamma':5.0,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 500,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.61024	eval-auc:0.60803
[10]	train-auc:0.63261	eval-auc:0.63752
[20]	train-auc:0.63756	eval-auc:0.64411
[30]	train-auc:0.64222	eval-auc:0.64910
[40]	train-auc:0.64608	eval-auc:0.65221
[50]	train-auc:0.64825	eval-auc:0.65332
[60]	train-auc:0.64994	eval-auc:0.65474
[70]	train-auc:0.65132	eval-auc:0.65510
[80]	train-auc:0.65148	eval-auc:0.65539
[90]	train-auc:0.65173	eval-auc:0.65538
[100]	train-auc:0.65346	eval-auc:0.65651
[110]	train-auc:0.65434	eval-auc:0.65687
[120]	train-auc:0.65552	eval-auc:0.65731
[130]	train-auc:0.65627	eval-auc:0.65787
[140]	train-auc:0.65745	eval-auc:0.65858
[150]	train-auc:0.65808	eval-auc:0.65905
[160]	train-auc:0.65870	eval-auc:0.65944
[170]	train-auc:0.65926	eval-auc:0.66021
[180]	train-auc:0.65995	eval-auc:0.66066
[190]	train-auc:0.66045	eval-auc:0.66088
[200]	train-auc:0.66124	eval-auc:0.66094
[210]	train-auc:0.66198	eval-auc:0.66133
[220]	train-auc:0.66228	eval-auc:0.66166
[230]	train-auc:0.66296	eval-auc:0.66231
[240]	train-auc:0.66315	eva

In [12]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.67, Gini: 0.35, KS: 0.25
Eval AUC: 0.66, Gini: 0.33, KS: 0.25
Test AUC: 0.66, Gini: 0.32, KS: 0.23
OOT AUC: 0.67, Gini: 0.33, KS: 0.25


In [13]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.374800         0.330853        0.551616        0.267357   1873   
9          0.251068         0.239637        0.267337        0.216982   1872   
8          0.197650         0.200502        0.216963        0.185074   1872   
7          0.181527         0.173306        0.185058        0.162558   1873   
6          0.177350         0.154252        0.162558        0.146387   1872   
5          0.134615         0.139765        0.146376        0.133950   1872   
4          0.125467         0.128471        0.133948        0.123135   1873   
3          0.107906         0.117465        0.123133        0.112249   1872   
2          0.077991         0.107391        0.112248        0.100636   1872   
1          0.053924         0.089966        0.100630        0.069632   1873   

        label_good  label_bad  
Decile                         
10            1171        702  
9             1402        470  
8             1502        370  
7             1533        340  
6             1540        332  
5             1620        252  
4             1638        235  
3             1670        202  
2             1726        146  
1             1772        101

In [14]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.356182         0.330875        0.538675        0.266904   1561   
9          0.251282         0.238449        0.266877        0.214648   1560   
8          0.221154         0.198830        0.214641        0.184504   1560   
7          0.180013         0.172136        0.184477        0.161671   1561   
6          0.162821         0.153180        0.161665        0.145748   1560   
5          0.146154         0.139447        0.145740        0.134178   1560   
4          0.121076         0.128468        0.134176        0.123007   1561   
3          0.111753         0.117568        0.123005        0.112170   1557   
2          0.085733         0.107082        0.112164        0.099987   1563   
1          0.073030         0.089549        0.099986        0.069632   1561   

        label_good  label_bad  
Decile                         
10            1005        556  
9             1168        392  
8             1215        345  
7             1280        281  
6             1306        254  
5             1332        228  
4             1372        189  
3             1383        174  
2             1429        134  
1             1447        114

In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.348722         0.328246        0.531942        0.265068   1213   
9          0.265458         0.238117        0.264908        0.216596   1213   
8          0.220940         0.199809        0.216579        0.184755   1213   
7          0.204620         0.173002        0.184747        0.162265   1212   
6          0.156636         0.153743        0.162226        0.146055   1213   
5          0.136026         0.139567        0.146013        0.134429   1213   
4          0.132013         0.129088        0.134387        0.123756   1212   
3          0.113768         0.118107        0.123734        0.112561   1213   
2          0.085738         0.108432        0.112511        0.102184   1213   
1          0.058533         0.091149        0.102173        0.069632   1213   

        label_good  label_bad  
Decile                         
10             790        423  
9              891        322  
8              945        268  
7              964        248  
6             1023        190  
5             1048        165  
4             1052        160  
3             1075        138  
2             1109        104  
1             1142         71

In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
18                                              g406s       190.0
13                         Age_of_oldest_open_account       162.0
6                highest_sanction_amount_credit_cards       147.0
1                        Num_Enq_all_loans_last_9M_3M       109.0
12                                    Utilization_all       104.0
15                         percentage_ever_delinquent        90.0
0                           Num_Enq_non_BANK_last_12M        90.0
2                            No of enq in past 90days        79.0
10          Utilization_PERSONAL LOAN_opened_last_12M        79.0
17                                     bankcard_vel_2        76.0
16                  num_CONSUMER LOAN_enq_last_6M_36M        64.0
5                 Num_of_PERSONAL LOAN_opened_last12M        33.0
9   Total_outstanding_Balance_PERSONAL LOAN_opened...        31.0
8                  num_PL_LE_1_Lac_opened_in_last_12M        29.0
4         

In [17]:
feature_importance_df

Feature  Importance
18                                              g406s       190.0
13                         Age_of_oldest_open_account       162.0
6                highest_sanction_amount_credit_cards       147.0
1                        Num_Enq_all_loans_last_9M_3M       109.0
12                                    Utilization_all       104.0
15                         percentage_ever_delinquent        90.0
0                           Num_Enq_non_BANK_last_12M        90.0
2                            No of enq in past 90days        79.0
10          Utilization_PERSONAL LOAN_opened_last_12M        79.0
17                                     bankcard_vel_2        76.0
16                  num_CONSUMER LOAN_enq_last_6M_36M        64.0
5                 Num_of_PERSONAL LOAN_opened_last12M        33.0
9   Total_outstanding_Balance_PERSONAL LOAN_opened...        31.0
8                  num_PL_LE_1_Lac_opened_in_last_12M        29.0
4                  Num_of_PERSONAL LOAN_opened_last9M        27.0
7                      num_CONSUMER LOAN_enq_last_36M        22.0
14                                          total_emi        16.0
3                      num_PERSONAL LOAN_enq_last_36M        10.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M         6.0

In [18]:
#Save the pred 
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BI_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BI_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BI_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BI_Oot_Pred_9024.csv',index=False)


In [19]:
# Save the model
xgb_model.save_model('LeoPayu_BI_90_in_24mob.json')
print('Model Saved')

Model Saved
